In [1]:
# transformer with phi3 is only available in latest version
!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.42.0.dev0
Uninstalling transformers-4.42.0.dev0:
  Successfully uninstalled transformers-4.42.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-82o4qbsy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-82o4qbsy
  Resolved https://github.com/huggingface/transformers to commit a564d10afe1a78c31934f0492422700f61a0ffc0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9130481 sha256=45021d500ff334372f4653fca1416aba89a6cfb9faba26b8b3807e9f1e93adb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zcte5h9/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [2]:
!pip install "pandas<2.0.0"

In [3]:
!pip install accelerate

  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (9

In [4]:
!pip install datasets

  Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


##  Import Packages

In [5]:
import pandas
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset, load_dataset, load_from_disk

In [6]:
# Setting torch seed
torch.random.manual_seed(0)

## Defining model & tokenizers

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [10]:
aspects = ["food", "service", "ambience", "price", "anecdotes/miscellaneous"]
sentiments = ["positive", "neutral", "negative"]


In [ ]:
messages = [
    {
        "role": "system",
        "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. The output should contain Aspects as keys and polarity of sentiment. Write 'None' as sentiment is aspect is not present in the input. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
    },
    {
        "role": "user",
        "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
    },
    {
        "role": "assistant",
        "content": "{'ambience': None,'food':'positive','other': None,'price':'positive','service':'positive',}",
    },
    {
        "role": "user",
        "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
    },
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(output[0]["generated_text"])

 {'ambience': None,'food': 'negative','price': 'neutral','service': None,}


## Loading data

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# Loading test data
dataset = load_from_disk("/content/drive/MyDrive/Scientific_project/dataset/test_dataset.hf")
dataset[0]

{'text': 'bread top notch well',
 'category': ['food'],
 'polarity': ['positive'],
 'joint': ['food#positive'],
 'category_labels': [0, 1, 0, 0, 0],
 'polarity_labels': [0, 0, 1],
 'joint_labels': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'true_labels': {'ambience': None,
  'food': 'positive',
  'other': None,
  'price': None,
  'service': None}}

### One shot

In [15]:
aspects = ["food", "service", "ambience", "price", "anecdotes/miscellaneous"]
sentiments = ["positive", "neutral", "negative"]


def prompting(data, prompt, prompt_id):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"{prompt}",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {prompt_id: output}

In [ ]:
prompt = f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation."
prompt_id = "prompt_1"


dataset_os = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=75
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [ ]:
dataset_os.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/results/phi3_prompt.csv"
)

In [40]:
prompt = f"You are a skilled data analyst. Your task is to extract and analyze sentiments from user reviews and structure the information into a JSON format. Analyze the provided user reviews and create a JSON summary. Focus on these key aspects: aspects = (food, service, ambience, price, anecdotes/miscellaneous)"
prompt_id = "prompt_2"



dataset_os2 = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=75
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [41]:
dataset_os2.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/results/phi4_prompt.csv"
)

In [16]:
prompt = f"You are an expert reviews annotator. You are performing aspect based sentiment analysis on a given restaurant review. Your task is to extract and analyze sentiments from user reviews and structure the information into a JSON format. "
prompt_id = "prompt_3"



dataset_os3 = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=75
)

Parameter 'function'=<function <lambda> at 0x7fd718143910> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [17]:
dataset_os3.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/results/phi5_prompt.csv"
)

In [ ]:
dataset_os3[0]

In [ ]:
prompt = f"  The aspects should only be {aspects} and sentiments should only be {sentiments}. The output should contain Aspects as keys and polarity of sentiment. Write 'None' as sentiment is aspect is not present in the input. Each aspects should only have one sentiment and not every aspect is necessarily present"